In [1]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))

import cudf
import cupy 
import numpy as np
import pandas as pd
import src.utils as utils
import src.calcEMA as calc_utils
import src.myenv as myenv


<built-in method __dir__ of sqlite3.Connection object at 0x7f5e2dde1740>


In [2]:
symbol = 'BTCUSDT'
interval = '1m'
start_amount_invested = 100.0
target_margin = 1.0
p_ema = 200
min_rsi = 28
max_rsi = 80
stop_loss_multiplier = 5

In [3]:
data = utils.get_data(
    symbol=symbol,
    interval=interval,
    updata_data_from_web=False,
	parse_dates=True)
#data = cudf.from_pandas(data)
data

,open_time,close
ix_open_time,,
2023-08-27 00:00:00,2023-08-27 00:00:00,26011.660156
2023-08-27 00:01:00,2023-08-27 00:01:00,26009.250000
2023-08-27 00:02:00,2023-08-27 00:02:00,26007.380859
2023-08-27 00:03:00,2023-08-27 00:03:00,26007.380859
2023-08-27 00:04:00,2023-08-27 00:04:00,26007.380859
...,...,...
2023-11-27 00:19:00,2023-11-27 00:19:00,37476.929688
2023-11-27 00:20:00,2023-11-27 00:20:00,37483.601562
2023-11-27 00:21:00,2023-11-27 00:21:00,37479.031250


In [4]:
# data = utils.truncate_data_in_days(data, myenv.days_to_validate_train)

In [5]:
data = calc_utils.calc_RSI(data)
data

,open_time,close,rsi
ix_open_time,,,
2023-08-27 00:00:00,2023-08-27 00:00:00,26011.660156,NaN
2023-08-27 00:01:00,2023-08-27 00:01:00,26009.250000,NaN
2023-08-27 00:02:00,2023-08-27 00:02:00,26007.380859,NaN
2023-08-27 00:03:00,2023-08-27 00:03:00,26007.380859,NaN
2023-08-27 00:04:00,2023-08-27 00:04:00,26007.380859,NaN
...,...,...,...
2023-11-27 00:19:00,2023-11-27 00:19:00,37476.929688,61.928013
2023-11-27 00:20:00,2023-11-27 00:20:00,37483.601562,63.612816
2023-11-27 00:21:00,2023-11-27 00:21:00,37479.031250,61.601784


In [6]:
data = calc_utils.calc_ema_periods(data, periods_of_time=[200])
data.dropna(inplace=True)
data

,open_time,close,rsi,ema_200p,ema_200p_diff
ix_open_time,,,,,
2023-08-27 03:19:00,2023-08-27 03:19:00,26012.099609,75.156357,26004.482422,0.029292
2023-08-27 03:20:00,2023-08-27 03:20:00,26018.509766,85.453949,26004.642578,0.053326
2023-08-27 03:21:00,2023-08-27 03:21:00,26018.519531,85.463837,26004.802734,0.052747
2023-08-27 03:22:00,2023-08-27 03:22:00,26018.529297,85.474464,26004.958984,0.052184
2023-08-27 03:23:00,2023-08-27 03:23:00,26018.529297,85.474464,26005.115234,0.051582
...,...,...,...,...,...
2023-11-27 00:19:00,2023-11-27 00:19:00,37476.929688,61.928013,37502.011719,-0.066882
2023-11-27 00:20:00,2023-11-27 00:20:00,37483.601562,63.612816,37501.828125,-0.048602
2023-11-27 00:21:00,2023-11-27 00:21:00,37479.031250,61.601784,37501.601562,-0.060185


In [7]:
_data = data

purchased = False
perform_sell = False
purchase_price = 0.0
purchase_strategy = ''
label_ema = f'ema_{p_ema}p'
start_amount_invested = 100.0

print(f'Start Simule Trading: {symbol}_{interval} - Shape: {_data.shape} - Amount Invested: {start_amount_invested:.2f} - Target Margin: {target_margin}% - EMA: {p_ema}p - Min RSI: {min_rsi}% - Max RSI: {max_rsi}% - Stop Loss Multiplier: {stop_loss_multiplier}')

for _, row in data.iterrows():
	#_auxdata = _data.iloc[row_nu:row_nu + 1]
	
	actual_price = row['close']
	rsi = row['rsi']
	p_ema_value = row[f'ema_{p_ema}p']
	open_time = row['open_time']
	#strategy = row['strategy']
	#take_profit = row['take_profit']
	#stop_loss = row['stop_loss']


	if isinstance(open_time, np.datetime64):
		_open_time = cudf.to_datetime(open_time, unit='ms').strftime('%Y-%m-%d %H:%M:%S')
	else:
		_open_time = open_time.strftime('%Y-%m-%d %H:%M:%S')
	
	strategy = utils.predict_strategy_index(row, p_ema, max_rsi, min_rsi)
#	print(f'[{_open_time}][{strategy}] => Purchased: {purchased} - Price: {actual_price:.6f} - RSI: {rsi:.2f} - ema_{p_ema}p: {p_ema_value:.2f} - Min RSI: {min_rsi} - Max RSI: {max_rsi}')
	if not purchased:
		if strategy.startswith('LONG') or strategy.startswith('SHORT'):  # If true, BUY
			purchased = True
			purchase_price = actual_price
			purchase_strategy = strategy
			take_profit, stop_loss = utils.calc_take_profit_stop_loss(strategy, purchase_price, target_margin, stop_loss_multiplier)
#			print(f'[{_open_time}] BUY: {purchase_strategy} - Price: {actual_price:.6f} - Target Margin: {target_margin:.2f}% - Take Profit: \
#{take_profit:.2f} - Stop Loss: {stop_loss:.2f} - RSI: {rsi:.2f}% - ema_{p_ema}p: {p_ema_value:.2f} - Min RSI: {min_rsi}% - Max RSI: {max_rsi}% - Stop Loss Multiplier: {stop_loss_multiplier}')
			continue

	if purchased:  # and (operation.startswith('LONG') or operation.startswith('SHORT')):
		# print(f'[{row_nu}][{strategy}] => Purchased: {purchased} - Price: {actual_price:.6f} - RSI: {rsi:.2f} - ema_{p_ema}p: {p_ema_value:.2f} - Min RSI: {min_rsi} - Max RSI: {max_rsi}')
		if purchase_strategy.startswith('LONG'):
			margin_operation = (actual_price - purchase_price) / purchase_price
			if ((actual_price >= take_profit) or (actual_price <= stop_loss)):  # Long ==> Sell - Take Profit / Stop Loss
				perform_sell = True
		elif purchase_strategy.startswith('SHORT'):
			margin_operation = (purchase_price - actual_price) / purchase_price
			if ((actual_price <= take_profit) or (actual_price >= stop_loss)):  # Short ==> Sell - Take Profit / Stop Loss
				perform_sell = True

		if perform_sell:
			start_amount_invested = (1 + margin_operation) * start_amount_invested
#			print(f'[{_open_time}] SELL: {purchase_strategy} - Price: {actual_price:.6f} - Purchase Price: {purchase_price:.6f} - Target Margin: {target_margin:.2f}% - Margin Operation:\
#{margin_operation*100:.2f}% - Take Profit: {take_profit:.2f} - Stop Loss: {stop_loss:.2f} - RSI: {rsi:.2f} - ema_{p_ema}p: {p_ema_value:.2f} - Sum PnL: {start_amount_invested:.2f}  - Stop Loss Multiplier: {stop_loss_multiplier}')
			perform_sell = False
			purchase_strategy = ''
			purchased = False

print(f'{symbol}_{interval} - Result Simule Trading: {start_amount_invested:.2f}')

Start Simule Trading: BTCUSDT_1m - Shape: (132305, 5) - Amount Invested: 100.00 - Target Margin: 1.0% - EMA: 200p - Min RSI: 28% - Max RSI: 80% - Stop Loss Multiplier: 5
BTCUSDT_1m - Result Simule Trading: 114.09


In [ ]:
#data = utils.predict_strategy_index_all_data(data, p_ema, min_rsi, max_rsi)
#data

In [ ]:
#data = utils.calc_take_profit_stop_loss_index(data, target_margin, stop_loss_multiplier)
#data

In [ ]:
strategy = utils.predict_strategy_index(data, p_ema, max_rsi, min_rsi)
print(strategy)

In [ ]:
for _, row in data.iterrows():
	strategy = utils.predict_strategy_index(row, p_ema, max_rsi, min_rsi)
	print(strategy)

In [ ]:
_data = data

def calc():
	purchased = False
	perform_sell = False
	purchase_price = 0.0
	purchase_strategy = ''
	label_ema = f'ema_{p_ema}p'
	start_amount_invested = 100.0

	print(f'Start Simule Trading: {symbol}_{interval} - Shape: {_data.shape} - Amount Invested: {start_amount_invested:.2f} - Target Margin: {target_margin}% - EMA: {p_ema}p - Min RSI: {min_rsi}% - Max RSI: {max_rsi}% - Stop Loss Multiplier: {stop_loss_multiplier}')

	for _, row in data.iterrows():
		#_auxdata = _data.iloc[row_nu:row_nu + 1]
		
		actual_price = row['close']
		rsi = row['rsi']
		p_ema_value = row[f'ema_{p_ema}p']
		open_time = row['open_time']
		strategy = row['strategy']
		take_profit = row['take_profit']
		stop_loss = row['stop_loss']

		if isinstance(open_time, np.datetime64):
			_open_time = pd.to_datetime(open_time, unit='ms').strftime('%Y-%m-%d %H:%M:%S')
		else:
			_open_time = open_time.strftime('%Y-%m-%d %H:%M:%S')

		if not purchased:
			# strategy = utils.predict_strategy_index(_auxdata, p_ema, max_rsi, min_rsi)
			# print(f'[{row_nu}][{strategy}] => Purchased: {purchased} - Price: {actual_price:.6f} - RSI: {rsi:.2f} - ema_{p_ema}p: {p_ema_value:.2f} - Min RSI: {min_rsi} - Max RSI: {max_rsi}')
			if strategy.startswith('LONG') or strategy.startswith('SHORT'):  # If true, BUY
				purchased = True
				purchase_price = actual_price
				purchase_strategy = strategy
				#take_profit, stop_loss = utils.calc_take_profit_stop_loss(strategy, purchase_price, target_margin, stop_loss_multiplier)
	#			print(f'BUY[{_open_time}]: {purchase_strategy} - Price: {actual_price:.6f} - Target Margin: {target_margin:.2f}% - Take Profit: \
	#{take_profit:.2f} - Stop Loss: {stop_loss:.2f} - RSI: {rsi:.2f}% - ema_{p_ema}p: {p_ema_value:.2f} - Min RSI: {min_rsi}% - Max RSI: {max_rsi}% - Stop Loss Multiplier: {stop_loss_multiplier}')
				continue

		if purchased:  # and (operation.startswith('LONG') or operation.startswith('SHORT')):
			# print(f'[{row_nu}][{strategy}] => Purchased: {purchased} - Price: {actual_price:.6f} - RSI: {rsi:.2f} - ema_{p_ema}p: {p_ema_value:.2f} - Min RSI: {min_rsi} - Max RSI: {max_rsi}')
			if purchase_strategy.startswith('LONG'):
				margin_operation = (actual_price - purchase_price) / purchase_price
				if ((actual_price >= take_profit) or (actual_price <= stop_loss)):  # Long ==> Sell - Take Profit / Stop Loss
					perform_sell = True
			elif purchase_strategy.startswith('SHORT'):
				margin_operation = (purchase_price - actual_price) / purchase_price
				if ((actual_price <= take_profit) or (actual_price >= stop_loss)):  # Short ==> Sell - Take Profit / Stop Loss
					perform_sell = True

			if perform_sell:
				start_amount_invested = (1 + margin_operation) * start_amount_invested
				#print(f'SELL[{_open_time}]: {purchase_strategy} - Price: {actual_price:.6f} - Purchase Price: {purchase_price:.6f} - Target Margin: {target_margin:.2f}% - Margin Operation:\
	#{margin_operation*100:.2f}% - Take Profit: {take_profit:.2f} - Stop Loss: {stop_loss:.2f} - RSI: {rsi:.2f} - ema_{p_ema}p: {p_ema_value:.2f} - Sum PnL: {start_amount_invested:.2f}  - Stop Loss Multiplier: {stop_loss_multiplier}')
				perform_sell = False
				purchase_strategy = ''
				purchased = False

	print(f'{symbol}_{interval} - Result Simule Trading: {start_amount_invested:.2f}')